# Molecular properties
## Structure optimization

One of the most commonly calculated properties in theoretical chemistry is the molecular structure. It is quite common to use structures calculated at a cheaper level of theory, typically DFT, as a starting point since ground state energies are relatively insensitive to the geometry. However, in some cases, especially when strong correlation is significant, one may want to use MCSCF structures instead.

The workflow here is very similar to that of VeloxChem, namely first defining the Gradient Driver, and then give it to the optimizer, which itself is based on GeomeTRIC. The MCSCF gradient driver will automatically choose analytical gradients if available for the specific type of MCSCF calculation performed, numerical gradients otherwise.

```{note}
Currently, derivative integrals are not available and all gradients are therefore numerical. This should change very soon.
```

Let's illustrate this on a simple O$_2$ molecule. First, we compute one MCSCF with the $\pi$ orbitals in the active space.

In [1]:
import veloxchem as vlx
import multipsi as mtp

O2_xyz="""2

O 0.0 0.0 -0.6
O 0.0 0.0  0.6
"""
molecule=vlx.Molecule.from_xyz_string(O2_xyz)
molecule.set_multiplicity(3)
basis = vlx.MolecularBasis.read(molecule,"cc-pvdz")

scfdrv = vlx.ScfUnrestrictedDriver()
scfdrv.compute(molecule, basis)
uhf_orbs = scfdrv.natural_orbitals()

space=mtp.OrbSpace(molecule,uhf_orbs)
space.cas(6,4)
mcscfdrv=mtp.McscfDriver()
mc_results = mcscfdrv.compute(molecule,basis,space)

                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Unrestricted Hartree-Fock                                       
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Error Vectors    : 10                                                                   
                

Now, we optimize the molecule.

In [2]:
opt_drv = mtp.OptimizationDriver(mcscfdrv)
opt_results = opt_drv.compute(molecule, basis, space)

                                                                                                                          
                                                Optimization Driver Setup                                                 
                                                                                                                          
                                     Coordinate System       :    TRIC                                                    
                                     Constraints             :    No                                                      
                                     Max. Number of Steps    :    300                                                     
                                     Transition State        :    No                                                      
                                     IRC                     :    No                                                      
                

We can now use this new geometry in subsequent calculations.

In [3]:
molecule = vlx.Molecule.read_xyz_string(opt_results['final_geometry'])
molecule.set_multiplicity(3)
mc_results = mcscfdrv.compute(molecule,basis,space)

                                                                                                                          
                          Multi-Configurational Self-Consistent Field Driver
                                                                                                                          

               Active space definition:
               ------------------------
               Number of inactive (occupied) orbitals: 5
               Number of active orbitals:              4
               Number of virtual orbitals:             19

               This is a CASSCF wavefunction: CAS(6,4)

               CI expansion:
               -------------
               Number of determinants:      6


                                                                                                                          
               ╭────────────────────────────────────╮
               │          Driver settings           │
               ╰──────────────

## Spectroscopy

### State-averaging

MCSCF is commonly used to simulate spectroscopy and photochemistry. There are several ways to compute excited state properties from a MCSCF calculation. As discussed in the previous section, one way is to use state-averaging:

Let's go back to our furan calculation. First we compute a SA-CASSCF with 5 states:

In [4]:
furan_xyz="""9

 C     -0.86213    -0.90784     0.00007
 H     -1.63433    -1.64264    -0.00003
 C      0.50727    -0.90524     0.00007
 C      0.92057     0.47886    -0.00003
 C     -0.22323     1.23186    -0.00003
 O     -1.35123     0.40376    -0.00013
 H      1.17117    -1.74724     0.00017
 H      1.93767     0.81866     0.00007
 H     -0.46573     2.26986    -0.00013
"""

molecule = vlx.Molecule.from_xyz_string(furan_xyz)
basis = vlx.MolecularBasis.read(molecule,"def2-sv(p)")

#We use a previously stored h5 file for the active space
space=mtp.OrbSpace(molecule,"furan-cas.h5")
mcscfdrv=mtp.McscfDriver()
mc_results = mcscfdrv.compute(molecule,basis,space, 5) #state-averaged with 5 states

                                                                                                                          
                          Multi-Configurational Self-Consistent Field Driver
                                                                                                                          

               Active space definition:
               ------------------------
               Number of inactive (occupied) orbitals: 15
               Number of active orbitals:              5
               Number of virtual orbitals:             58

               This is a CASSCF wavefunction: CAS(6,5)

               CI expansion:
               -------------
               Number of determinants:      100


                                                                                                                          
               ╭────────────────────────────────────╮
               │          Driver settings           │
               ╰───────────

Now we can compute the transition properties using the InterState module. This module requires the dictionary returned by the MCSCF.

In [5]:
#Compute oscillator strengths
SI=mtp.StateInteraction()
Prop_dict=SI.compute(molecule,basis, mc_results)

                                                                                                                          
List of oscillator strengths greather than 1e-10
                                                                                                                          
  From     to       Energy (eV)    Oscillator strength (length and velocity)
     1       2        6.73075         3.538251e-03    1.585306e-03
     1       3        8.16775         2.119360e-01    1.437357e-01
     1       4        9.96698         6.215234e-01    4.429841e-01
     1       5       10.90222         4.134625e-01    2.851360e-01
                                                                                                                          
List of rotatory strengths greather than 1e-10
                                                                                                                          
  From     to       Energy (eV)    Rot. strength (a.u. and 10^-40 c

The function returns a dictionary with the most important values.

In [6]:
print(Prop_dict["energies"])
print(Prop_dict["oscillator_strengths"])

[0.24735066 0.30015917 0.36627994 0.40064915]
[0.00353825 0.21193598 0.62152339 0.41346248]


By default InterState computes properties from the first state to all others, but this can be changed by providing the list of "initial" and "final" states as arguments.

In [7]:
Prop_dict=SI.compute(molecule,basis,mc_results, from_states=[0,1], to_states='all' )

                                                                                                                          
List of oscillator strengths greather than 1e-10
                                                                                                                          
  From     to       Energy (eV)    Oscillator strength (length and velocity)
     1       2        6.73075         3.538251e-03    1.585306e-03
     1       3        8.16775         2.119360e-01    1.437357e-01
     1       4        9.96698         6.215234e-01    4.429841e-01
     1       5       10.90222         4.134625e-01    2.851360e-01
     2       1       -6.73075        -3.538251e-03   -1.585306e-03
     2       3        1.43699         2.336494e-02    1.580513e-02
     2       4        3.23623         1.119135e-03    3.834218e-04
     2       5        4.17146         9.031471e-02    8.060789e-02
                                                                                            

### Linear response

Another way to compute excited state properties at the MCSCF level is using linear response. This multiconfigurational linear response (MCLR) requires first a MCSCF calculation of the ground state and then the use of the Mclr_EigenSolver module:

In [8]:
mcscfdrv=mtp.McscfDriver()
mc_results = mcscfdrv.compute(molecule,basis,space) #only ground state!

mcrpa = mtp.Mclr_EigenSolver()
Prop_dict = mcrpa.compute(molecule, basis, mcscfdrv, n_states = 2)

                                                                                                                          
                          Multi-Configurational Self-Consistent Field Driver
                                                                                                                          

               Active space definition:
               ------------------------
               Number of inactive (occupied) orbitals: 15
               Number of active orbitals:              5
               Number of virtual orbitals:             58

               This is a CASSCF wavefunction: CAS(6,5)

               CI expansion:
               -------------
               Number of determinants:      100


                                                                                                                          
               ╭────────────────────────────────────╮
               │          Driver settings           │
               ╰───────────

Again, the Prop_dict dictionary gives us access to the most important values.

An advantage of linear response is that unlike state-averaging, the excitation energies does not depend on the number of calculated states (except when the code accidentally converges to a higher solution, in which case increasing the number of states tends to ensure we get actually the lowest ones). Often, linear response provides more accurate excitation energies than state-averaging, the main exception being close to crossing point between the ground and excited states.

Note that currently only the Tamm-Dancoff approximation is available (using the property "mcrpa.tda = True") which gives a slightly faster but less accurate calculation.